# Agents

## Initialize agents 

In [57]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENWEATHERMAP_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")


In [58]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper

weather = OpenWeatherMapAPIWrapper()

In [59]:
weather_data = weather.run("Schweinfurt")
print(weather_data)

In Schweinfurt, the current weather is as follows:
Detailed status: few clouds
Wind speed: 5.25 m/s, direction: 289°
Humidity: 80%
Temperature: 
  - Current: 6.65°C
  - High: 7.4°C
  - Low: 5.25°C
  - Feels like: 3.29°C
Rain: {}
Heat index: None
Cloud cover: 12%


In [60]:
type(weather_data)

str

In [61]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI


llm = OpenAI(temperature=0)

tools = load_tools(["openweathermap-api"], llm)

agent_chain = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [62]:
agent_chain.run("What's the weather like in Schweinfurt?")



> Entering new AgentExecutor chain...
 I should use the OpenWeatherMap tool to fetch current weather information for a specified location.
Action: OpenWeatherMap
Action Input: Schweinfurt
Observation: In Schweinfurt, the current weather is as follows:
Detailed status: few clouds
Wind speed: 5.25 m/s, direction: 289°
Humidity: 80%
Temperature: 
  - Current: 6.65°C
  - High: 7.4°C
  - Low: 5.25°C
  - Feels like: 3.29°C
Rain: {}
Heat index: None
Cloud cover: 12%
Thought: I now know the final answer.
Final Answer: The current weather in Schweinfurt is few clouds with a temperature of 6.65°C.

> Finished chain.


'The current weather in Schweinfurt is few clouds with a temperature of 6.65°C.'

## Custom agent example

In [102]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_openai import ChatOpenAI


In [103]:
model = ChatOpenAI()

In [113]:
@tool
def search(query: str) -> str:
    """Search things about current events."""
    return " 32 degrees"

In [114]:
tool_list = [search]

In [115]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/xml-agent-convo")

In [116]:
# Logic for going from intermediate steps to a string to pass into model
# This is pretty tied to the prompt
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return f"<intermediate_steps>{log}</intermediate_steps>"


# Logic for converting tools to string to go in prompt
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

In [117]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: convert_intermediate_steps(
            x["intermediate_steps"]
        ),
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgentOutputParser()
)

In [118]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

In [119]:
agent_executor.invoke({"input": "whats the weather in Berlin?"})



> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>weather in Berlin 32 degrees<final_answer>The weather in Berlin is 32 degrees.

> Finished chain.


{'input': 'whats the weather in Berlin?',
 'output': 'The weather in Berlin is 32 degrees.'}